#### Change note:
1. judge if winningHighlight is NaN. If NaN, each word becomes a segment since NaN(currentType)!=NaN(lastType). Earlier we have 49050 segments for 2071 papers, now we have 19702 segments for 2071 papers. 
2. Catch the last segment, currently we save until the second last segments whose winningHighlight is different from lastType. This ignores the N-th segment.
3. (Also reflected in test run 8) Ideally we want to have # dev_segments = # line count in annotations_label-level_all-to-date-2018-4-25-WithTitle.labelled.originalSegments_level_1.csv. Likewise for # test_segments = # line count in ..._level_2.csv. However, we found the count does not match. The reasons is that in make-pseudo-docs.ipynb. Check the code,
    
    `for split in row['content'].split('.'):`

Above line splits word-concatenated order-preserving sequences for a segment with . (period), but not ?(question mark) or !(exclamation). TODO: This could be fixed later.

In [119]:
import pandas as pd
import pathlib
import numpy as np
import csv

In [146]:
level=1
filter_paper_dataset=pathlib.Path.cwd().joinpath('data', 'Filtered_MLD_RI_HCI_CSD_LTI_ISR.csv')

docid_prefix="2K_test_"
 
if level==1:
	with open(filter_paper_dataset) as csvfile:
		reader=csv.reader(csvfile)
		next(reader)

		paperID_set=set([r[26].strip() for r in reader]) # paper ID column is 26
	docid_prefix="2K_dev_"
    
print(list(paperID_set)[:10])  
print("# of unique title set ", len(set(paperID_set)))

['msa_2613209260', 'msa_2098790909', 'msa_2286742647', 'msa_2026037005', 'msa_2213606837', 'msa_2319135850', 'msa_2530103166', 'msa_81017276', 'msa_2583283942', 'msa_1979765269']
# of unique title set  497


Load the data. We want to use the words level file, since it preserves the original sentence ordering. NOTE: might take a bit of time to load (more than a few seconds), since the file is quite large.

In [121]:
words_path = pathlib.Path.cwd().joinpath('data', 'annotations_word-level_all-to-date-2018-4-25.xlsx')

In [122]:
words = pd.read_excel(words_path)

In [123]:
words.count()

_id                     387548
content                 387525
docID                   387548
globalPsn               387548
highlightsBackground    387548
highlightsFindings      387548
highlightsMechanism     387548
highlightsMethod        387548
highlightsPurpose       387548
sentenceID              387548
sequence                387548
winningHighlight        349903
paperID                 387548
dtype: int64

In [124]:
words[words['paperID']=='wos_cs_507']

,_id,content,docID,globalPsn,highlightsBackground,highlightsFindings,highlightsMechanism,highlightsMethod,highlightsPurpose,sentenceID,sequence,winningHighlight,paperID
81018,59dcb9732bdc9e7b0a6ff4b9,This,59dcb9732bdc9e7b0a6ff4b7,1,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,1,NaN,wos_cs_507
81019,59dcb9732bdc9e7b0a6ff4ba,invited,59dcb9732bdc9e7b0a6ff4b7,2,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,2,NaN,wos_cs_507
81020,59dcb9732bdc9e7b0a6ff4bb,talk,59dcb9732bdc9e7b0a6ff4b7,3,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,3,NaN,wos_cs_507
81021,59dcb9732bdc9e7b0a6ff4bc,provides,59dcb9732bdc9e7b0a6ff4b7,4,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,4,NaN,wos_cs_507
81022,59dcb9732bdc9e7b0a6ff4bd,a,59dcb9732bdc9e7b0a6ff4b7,5,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,5,NaN,wos_cs_507
81023,59dcb9732bdc9e7b0a6ff4be,brief,59dcb9732bdc9e7b0a6ff4b7,6,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,6,NaN,wos_cs_507
81024,59dcb9732bdc9e7b0a6ff4bf,exposition,59dcb9732bdc9e7b0a6ff4b7,7,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,7,NaN,wos_cs_507
81025,59dcb9732bdc9e7b0a6ff4c0,how,59dcb9732bdc9e7b0a6ff4b7,8,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,8,NaN,wos_cs_507
81026,59dcb9742bdc9e7b0a6ff4c1,hybrid,59dcb9732bdc9e7b0a6ff4b7,9,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,9,NaN,wos_cs_507
81027,59dcb9742bdc9e7b0a6ff4c2,systems,59dcb9732bdc9e7b0a6ff4b7,10,[],[],[],[],[],59dcb9732bdc9e7b0a6ff4b8,10,NaN,wos_cs_507


In [147]:
print("There are", len(set(words['paperID'])), "unique docs")

There are 2071 unique docs


Then I think we want to group by paperID, make sure that it's sorted by globalPsn, and then make:
1. A new csv that has a coherent segment as a row, with order preserved.
2. Pseudo-docs that join these rows with a "fake" separator that Snorkel can use to split into candidates.

In [148]:
validTypes = set(['Method', 'Background', 'Findings', 'Mechanism', 'Purpose'])

In [149]:
print(set(words['winningHighlight']))

{nan, 'Purpose', 'Findings', 'Method', 'Background', 'None', 'Mechanism'}


In [150]:
segmented = []
import math

recalled_doc_count=0
recalled_doc_set=set()
for paperID, paperData in words.groupby('paperID'):
    if level==1 and paperID.strip() not in paperID_set:
        print(paperID, "NOT in title_set")
        continue
    else:
        if level==1:
            recalled_doc_count+=1
            recalled_doc_set.add(paperID.strip())
        pass
    
    # make sure everything is sorted first
    paperData = paperData.sort_values(by="globalPsn")
    
    # get the first row first
    # we're going to switch if the current highlight is different from the last one
    firstRow = paperData.iloc[0]
    lastType = firstRow['winningHighlight']
    segmentWords = [str(firstRow['content'])]
    sequence = 1
    #secondRowOnwards = paperData.iloc[1:]
    
    for index, row in paperData.iloc[1:].iterrows():
        
        # get currentType
        currentType = row['winningHighlight']
#         if paperID=="wos_cs_507":
#             print(currentType,lastType)
        # propagate past empty ones (this is mostly the periods)
        if currentType not in validTypes:
            currentType = lastType
        
        # compare with lastType
        if currentType != lastType and not currentType!=currentType :  # nan is not self-equal, this is a way to check if currentType is NaN
            # we've switched types
            # process the last segment group
            joined = " ".join(segmentWords)
            segmented.append({
                'content': joined,
                'highlightType': lastType,
                'paperID': docid_prefix+paperID,
                'sequence': sequence
            })
            # start a new segment group
            segmentWords = [str(row['content'])]
            sequence += 1
        else:
            # we're still in the same type
            # add to the current segment group
            segmentWords.append(str(row['content']))
            
        # set lastType for next comparison
        lastType = currentType
    if len(segmentWords)>0:
        joined = " ".join(segmentWords)
        segmented.append({
            'content': joined,
            'highlightType': lastType,
            'paperID': docid_prefix+paperID,
            'sequence': sequence
        })
        
segmented = pd.DataFrame(segmented)
segmented.head(25)

print(recalled_doc_count)

print(paperID_set.difference(recalled_doc_set))

2017_1 NOT in title_set
2017_10 NOT in title_set
2017_11 NOT in title_set
2017_4 NOT in title_set
2017_5 NOT in title_set
2017_6 NOT in title_set
2017_7 NOT in title_set
2017_8 NOT in title_set
2017_9 NOT in title_set
356QDHJ7 NOT in title_set
4C8T6PTV NOT in title_set
4IR7FP3G NOT in title_set
56QUJEMP NOT in title_set
5G8U33QZ NOT in title_set
6UGSWD8D NOT in title_set
7V9JCFSV NOT in title_set
8M9FQCNU NOT in title_set
9895DC6Q NOT in title_set
9T2T3NWW NOT in title_set
ABX4R7K5 NOT in title_set
AFWXF8CH NOT in title_set
AMDVNRNA NOT in title_set
CDAJEXM8 NOT in title_set
CZQ57NCZ NOT in title_set
DW9W8Z9I NOT in title_set
EWDTT8PI NOT in title_set
F5SXURFA NOT in title_set
F96H6I58 NOT in title_set
FQVMU8GU NOT in title_set
GCGB74CJ NOT in title_set
GPBS8H6W NOT in title_set
GQPNFE6B NOT in title_set
H3HIEFKV NOT in title_set
HTFNIQDD NOT in title_set
J7SB3TH5 NOT in title_set
KF3CZPHV NOT in title_set
MXXBE2S3 NOT in title_set
NINP5I8I NOT in title_set
NX42RPT2 NOT in title_set
QV

msa_2025945592 NOT in title_set
msa_2026290661 NOT in title_set
msa_2026522435 NOT in title_set
msa_2027670258 NOT in title_set
msa_2028257008 NOT in title_set
msa_2028418570 NOT in title_set
msa_2028502081 NOT in title_set
msa_2028637253 NOT in title_set
msa_2028977569 NOT in title_set
msa_2029030079 NOT in title_set
msa_2030012418 NOT in title_set
msa_2030245620 NOT in title_set
msa_203052366 NOT in title_set
msa_2030541971 NOT in title_set
msa_2030904677 NOT in title_set
msa_2031325092 NOT in title_set
msa_2031471786 NOT in title_set
msa_2031503222 NOT in title_set
msa_2031535007 NOT in title_set
msa_2032342062 NOT in title_set
msa_2032695641 NOT in title_set
msa_2032987408 NOT in title_set
msa_2033396088 NOT in title_set
msa_2033552406 NOT in title_set
msa_2033760099 NOT in title_set
msa_2034078512 NOT in title_set
msa_2035874553 NOT in title_set
msa_2036131936 NOT in title_set
msa_2037352755 NOT in title_set
msa_2037429787 NOT in title_set
msa_2038943390 NOT in title_set
msa_20389

msa_2175595662 NOT in title_set
msa_2176615452 NOT in title_set
msa_2176928290 NOT in title_set
msa_2177412666 NOT in title_set
msa_2178355895 NOT in title_set
msa_2179541718 NOT in title_set
msa_2179754069 NOT in title_set
msa_2182420680 NOT in title_set
msa_2183629688 NOT in title_set
msa_2184450955 NOT in title_set
msa_2195367229 NOT in title_set
msa_2200002159 NOT in title_set
msa_2200914098 NOT in title_set
msa_2200922230 NOT in title_set
msa_2202849354 NOT in title_set
msa_2203400881 NOT in title_set
msa_2203738484 NOT in title_set
msa_2203912317 NOT in title_set
msa_2204075824 NOT in title_set
msa_2205026508 NOT in title_set
msa_2207475308 NOT in title_set
msa_2209204668 NOT in title_set
msa_2209394967 NOT in title_set
msa_2209405713 NOT in title_set
msa_2210583849 NOT in title_set
msa_2212199686 NOT in title_set
msa_2213682550 NOT in title_set
msa_2215375112 NOT in title_set
msa_2219743133 NOT in title_set
msa_2220556757 NOT in title_set
msa_2226721295 NOT in title_set
msa_2230

msa_2566221905 NOT in title_set
msa_2566861726 NOT in title_set
msa_2568091325 NOT in title_set
msa_2568772110 NOT in title_set
msa_2569001686 NOT in title_set
msa_2570678867 NOT in title_set
msa_2572434179 NOT in title_set
msa_2572789406 NOT in title_set
msa_2573285109 NOT in title_set
msa_2573586242 NOT in title_set
msa_2573721906 NOT in title_set
msa_2575532943 NOT in title_set
msa_2575586770 NOT in title_set
msa_2576668423 NOT in title_set
msa_2576845151 NOT in title_set
msa_2576902070 NOT in title_set
msa_2578908617 NOT in title_set
msa_2579157225 NOT in title_set
msa_2579730587 NOT in title_set
msa_2580548319 NOT in title_set
msa_2580899942 NOT in title_set
msa_2581837924 NOT in title_set
msa_2582449424 NOT in title_set
msa_2582553469 NOT in title_set
msa_2583888926 NOT in title_set
msa_2584769872 NOT in title_set
msa_2585051807 NOT in title_set
msa_2585125256 NOT in title_set
msa_2585370664 NOT in title_set
msa_2585454065 NOT in title_set
msa_2585559133 NOT in title_set
msa_2585

wos_cs_248 NOT in title_set
wos_cs_249 NOT in title_set
wos_cs_250 NOT in title_set
wos_cs_251 NOT in title_set
wos_cs_252 NOT in title_set
wos_cs_253 NOT in title_set
wos_cs_254 NOT in title_set
wos_cs_256 NOT in title_set
wos_cs_257 NOT in title_set
wos_cs_259 NOT in title_set
wos_cs_26 NOT in title_set
wos_cs_261 NOT in title_set
wos_cs_264 NOT in title_set
wos_cs_265 NOT in title_set
wos_cs_266 NOT in title_set
wos_cs_267 NOT in title_set
wos_cs_270 NOT in title_set
wos_cs_271 NOT in title_set
wos_cs_273 NOT in title_set
wos_cs_275 NOT in title_set
wos_cs_277 NOT in title_set
wos_cs_279 NOT in title_set
wos_cs_28 NOT in title_set
wos_cs_281 NOT in title_set
wos_cs_282 NOT in title_set
wos_cs_285 NOT in title_set
wos_cs_287 NOT in title_set
wos_cs_289 NOT in title_set
wos_cs_29 NOT in title_set
wos_cs_291 NOT in title_set
wos_cs_295 NOT in title_set
wos_cs_297 NOT in title_set
wos_cs_298 NOT in title_set
wos_cs_30 NOT in title_set
wos_cs_300 NOT in title_set
wos_cs_301 NOT in title_

Ok let's check a few to see what segments come out!

In [151]:
for paperID, paperData in segmented.head(25).groupby("paperID"):
    print("Paper:", paperID)
    print("*"*30)
    paperData = paperData.sort_values(by="sequence")
    for index, row in paperData.iterrows():
        print(row['highlightType'], row['content'])

Paper: 2K_dev_msa_105909713
******************************
Background Languages for music audio processing typically offer a large assortment of unit generators . There is great duplication among different language implementations , as each language must implement many of the same ( or nearly the same ) unit generators .
Purpose Csound has a large library of unit generators and could be a useful source of reusable unit generators for other languages or for direct use in applications . In this study , we consider how Csound unit generators can be exposed to direct access by other audio processing languages .
Mechanism Using Aura as an example , we modified Csound to allow efficient , dynamic allocation of individual unit generators without using the Csound compiler or writing Csound instruments . We then extended Aura using automatic code generation so that Csound unit generators can be accessed in the normal way from within Aura . In this scheme , Csound details are completely hidden f

Now let's create Snorkel-style segments, split by periods.

In [152]:
import re
segmented_periodSplit = []
for paperID, segmentedPaper in segmented.groupby("paperID"):
    sequence = 1
    for index, row in segmentedPaper.iterrows():
        for split in re.split(r"\.|\?|!",row['content']): #.split('.|?|!'):
            if len(split) > 1: # skip blank content
                segmented_periodSplit.append({
                    'content': split.strip(), # remove leading/trailing spaces from the period split
                    'highlightType': row['highlightType'],
                    'paperID': paperID,
                    'sequence': sequence
                })
            sequence += 1
segmented_periodSplit = pd.DataFrame(segmented_periodSplit)

In [153]:
segmented_periodSplit.head(50)

,content,highlightType,paperID,sequence
0,Languages for music audio processing typically...,Background,2K_dev_msa_105909713,1
1,There is great duplication among different lan...,Background,2K_dev_msa_105909713,2
2,Csound has a large library of unit generators ...,Purpose,2K_dev_msa_105909713,4
3,"In this study , we consider how Csound unit ge...",Purpose,2K_dev_msa_105909713,5
4,"Using Aura as an example , we modified Csound ...",Mechanism,2K_dev_msa_105909713,7
5,We then extended Aura using automatic code gen...,Mechanism,2K_dev_msa_105909713,8
6,"In this scheme , Csound details are completely...",Mechanism,2K_dev_msa_105909713,9
7,We suggest that these techniques might elimina...,Background,2K_dev_msa_105909713,11
8,Motivated by a radically new peer review syste...,Background,2K_dev_msa_1129480147,1
9,we study peer review systems in which proposal...,Purpose,2K_dev_msa_1129480147,2


In [154]:
out_path = pathlib.Path.cwd().joinpath('data', 'annotations_label-level_all-to-date-2018-4-25-WithTitle.labelled.originalSegments_level_'+str(level)+'.csv')
segmented_periodSplit.to_csv(out_path, columns=['content', 'highlightType', 'paperID'], index=False, header=False)